In [1]:
import pandas as pd
from keras.layers import Conv1D, GRU, Dense, Dropout, Flatten
from keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
cleared_data_all = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset.xlsx')
cleared_data_all = cleared_data_all.iloc[:, 1:]
cleared_data_all

,Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (current US$)","Adjusted net savings, including particulate emission damage (current US$)",Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (current US$),...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.541305,0.018580,0.001411,0.493338,0.255733,0.024477,0.042788,0.041910,7.641349e-07,0.000160,...,0.109494,0.006858,0.098921,0.047494,0.284959,0.366300,0.211988,0.381785,0.440389,0.443970
1,0.542030,0.011485,0.001591,0.497128,0.257579,0.031381,0.043985,0.044182,1.141130e-04,0.000511,...,0.061274,0.036062,0.090457,0.054871,0.267499,0.335687,0.295060,0.331167,0.353970,0.359865
2,0.532645,0.013996,0.002015,0.486800,0.309672,0.028568,0.042789,0.046088,4.111755e-05,0.000642,...,0.056934,0.048997,0.112458,0.038223,0.235780,0.293609,0.236735,0.276357,0.283570,0.325602
3,0.529905,0.021782,0.001641,0.513873,0.335287,0.027832,0.042536,0.043784,1.076366e-04,0.000626,...,0.073445,0.042937,0.100149,0.045912,0.309495,0.269240,0.292649,0.269380,0.402339,0.451430
4,0.523391,0.015874,0.001086,0.505739,0.319546,0.025767,0.042422,0.048538,1.100043e-04,0.000375,...,0.052960,0.027781,0.075377,0.038749,0.300016,0.216947,0.173194,0.303925,0.367756,0.463100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.430820,0.016480,0.019676,0.381147,0.222597,0.355406,0.037475,0.030756,3.170770e-02,0.018601,...,0.598647,0.633237,0.791004,0.608203,0.052758,0.094311,0.103817,0.493570,0.648812,0.429089
2028,0.431366,0.016379,0.015311,0.380339,0.222985,0.235515,0.037528,0.030807,2.144808e-02,0.015013,...,0.581060,0.480167,0.845445,0.906866,0.054837,0.096322,0.085744,0.495072,0.788197,0.621088
2029,0.430798,0.016389,0.017261,0.382386,0.324990,0.292323,0.037512,0.030619,2.867491e-02,0.019105,...,0.652924,0.659372,0.775232,0.769994,0.060454,0.098751,0.085432,0.472306,0.549884,0.540978
2030,0.430458,0.016307,0.023369,0.381875,0.328690,0.283576,0.037511,0.030659,3.792288e-02,0.022233,...,0.623938,0.697836,0.750916,0.541441,0.071787,0.080619,0.092265,0.654687,0.516524,0.400289


In [4]:
cleared_data_only_percent = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset_only_%.xlsx')
cleared_data_only_percent = cleared_data_only_percent.iloc[:, 1:]
cleared_data_only_percent

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.553663,0.493048,0.550624,0.659424,0.482240,0.204614,0.370799,0.527068,1.000000,0.332565,...,0.092583,0.044941,0.116194,0.058954,0.271638,0.256557,0.236409,0.340039,0.213734,0.288946
1,0.540093,0.520842,0.572925,0.515892,0.579728,0.170912,0.409697,0.533898,0.949101,0.361794,...,0.080344,0.045471,0.119967,0.067618,0.240226,0.257459,0.168916,0.451473,0.356074,0.258655
2,0.535596,0.540533,0.599002,0.593352,0.517354,0.204310,0.433730,0.517269,0.943975,0.278636,...,0.073603,0.050711,0.072459,0.041396,0.288064,0.289370,0.211341,0.448748,0.419979,0.258733
3,0.526273,0.508690,0.638806,0.523286,0.468628,0.196385,0.342814,0.533692,0.838995,0.384575,...,0.077985,0.041490,0.144534,0.034277,0.245759,0.284184,0.213594,0.461372,0.347358,0.254320
4,0.549766,0.500511,0.690262,0.515493,0.394821,0.132378,0.330198,0.532333,0.961634,0.409441,...,0.068681,0.047989,0.115199,0.022896,0.237920,0.279386,0.248200,0.383103,0.306070,0.305546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.422859,0.408177,0.546620,0.209895,0.347769,0.502068,0.172372,0.378021,0.333544,0.478746,...,0.842023,0.675799,0.880286,0.653719,0.101137,0.068454,0.085621,0.401947,0.498277,0.469291
2028,0.422482,0.407757,0.569233,0.023101,0.358500,0.622407,0.193755,0.358328,0.317363,0.507215,...,0.957228,0.608725,0.816423,0.536649,0.112019,0.073421,0.059600,0.339091,0.414780,0.576167
2029,0.422558,0.409683,0.625547,0.099384,0.356030,0.667137,0.170789,0.352184,0.282537,0.640014,...,0.841818,0.642024,0.733566,0.716802,0.087615,0.078230,0.059868,0.450768,0.445414,0.564287
2030,0.423154,0.409955,0.650208,0.126015,0.461726,0.585527,0.200973,0.345227,0.228218,0.798934,...,0.757523,0.588630,0.698973,0.602522,0.068091,0.081960,0.074794,0.579578,0.441237,0.553940


Division into training and test samples

In [5]:
test_size = 0.2
seed = 7
target_feature_name = 'GDP per capita (current US$)'

data_X_all = cleared_data_all.drop([target_feature_name], axis=1)
data_Y_all = cleared_data_all[target_feature_name]
data_X_train_all, data_X_test_all, data_Y_train_all, data_Y_test_all = train_test_split(data_X_all, data_Y_all, test_size=test_size, random_state=seed)

data_X_only_percent = cleared_data_only_percent.drop([target_feature_name], axis=1)
data_Y_only_percent = cleared_data_only_percent[target_feature_name]
data_X_train_only_percent, data_X_test_only_percent, data_Y_train_only_percent, data_Y_test_only_percent = train_test_split(data_X_only_percent, data_Y_only_percent, test_size=test_size, random_state=seed)

datasets = [{'X_train': data_X_train_only_percent, 'X_test': data_X_test_only_percent, 'Y_train': data_Y_train_only_percent,
             'Y_test': data_Y_test_only_percent, 'name': 'only_percent_dataset'},
            {'X_train': data_X_train_all, 'X_test': data_X_test_all,
             'Y_train': data_Y_train_all, 'Y_test':data_Y_test_all, 'name': 'original_data'}]

In [6]:
datasets[0].get('X_test')

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
1997,0.424236,0.408247,0.427053,0.107310,0.180478,0.424943,0.169467,0.481029,0.431075,0.435744,...,0.636536,0.662121,0.695450,0.638801,0.069544,0.069428,0.054219,0.666814,0.501677,0.407341
1473,0.447738,0.460390,0.610356,0.138927,0.272227,0.349601,0.200741,0.535050,0.660656,0.432643,...,0.146932,0.204988,0.170579,0.187240,0.045789,0.059786,0.049883,0.554295,0.573229,0.706630
1780,0.446317,0.427323,0.674869,0.217281,0.113438,0.239320,0.069061,0.524442,0.474822,0.213609,...,0.439474,0.599492,0.717330,0.690639,0.088564,0.033210,0.072647,0.730002,0.515049,0.503232
1549,0.528160,0.534537,0.598327,0.275627,0.393298,0.630570,0.075976,0.500581,0.586302,0.407304,...,0.147340,0.096598,0.136002,0.158403,0.039660,0.036211,0.046145,0.529084,0.398535,0.834956
386,0.414741,0.409854,0.494516,0.460125,0.511808,0.462622,0.257173,0.485036,0.451292,0.628560,...,0.230779,0.233867,0.366804,0.244210,0.196414,0.169540,0.162719,0.532888,0.225868,0.250416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0.445155,0.430177,0.491887,0.319256,0.444357,0.250446,0.352527,0.598689,0.473074,0.557277,...,0.121806,0.110419,0.153444,0.146445,0.213777,0.253562,0.232608,0.455449,0.301778,0.385986
98,0.419985,0.383730,0.473309,0.330835,0.264762,0.417222,0.546214,0.464626,0.474719,0.280847,...,0.106647,0.035363,0.054978,0.034675,0.219160,0.346347,0.180789,0.270916,0.298001,0.390620
768,0.487644,0.486324,0.560683,0.279337,0.711198,0.114109,0.011724,0.918513,0.670251,0.323551,...,0.073287,0.029154,0.100611,0.032287,0.362384,0.309156,0.376537,0.394202,0.212536,0.317965
610,0.532147,0.525056,0.577301,0.366375,0.278995,0.359277,0.169314,0.876354,0.775569,0.396121,...,0.035026,0.030594,0.036447,0.011276,0.402965,0.539792,0.372430,0.227441,0.150807,0.314956


Serial connection one-dimensional convolutional and recurrent GRU neural network layers with a fully connected classifier

In [7]:
param_grid_neural_network = {
    'filters': [8, 16, 32],
    'kernel_size': [1, 2, 3],
    'units': [16, 32, 64],
    'optimizer' : ["adam", "adadelta", "rmsprop"],
    'dropout_rate': [0.1, 0.2, 0.3],
    'gru_act': ['tanh', 'relu'],
    'conv_act': ['tanh', 'relu']
}

In [ ]:
def create_model(filters, kernel_size, units, dropout_rate, optimizer_model, gru_act, conv_act):
    input_shape = (dataset.get('X_train').shape[1], 1)
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=conv_act, input_shape=input_shape))
    model.add(GRU(units, input_shape=input_shape, activation=gru_act))
    model.add(Dropout(rate=dropout_rate))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=optimizer_model, loss='mean_squared_error')
    return model

best_params = []
mse = []

for dataset in datasets:
    regressor = KerasRegressor(build_fn=create_model, filters=32, kernel_size=3, units=64, dropout_rate=0.2, optimizer_model='adam', gru_act='tanh', conv_act='relu', epochs=3, batch_size=32)
    grid = GridSearchCV(estimator=regressor, param_grid=param_grid_neural_network, n_jobs=7)
    grid_result = grid.fit(dataset.get('X_train'), dataset.get('Y_train'), validation_data = (dataset.get('X_test'), dataset.get('Y_test')))

    best_params.append(grid_result.best_params_)
    print("Лучшее значение параметров: ", grid_result.best_params_)

    score = grid.score(dataset.get('X_test'), dataset.get('Y_test'))
    mse.append(score)
    print("MSE:", score)

print(best_params)
print(mse)